# Reading Data from MongoDB using API Key

In [8]:
import requests
import json

with open("secrets.json", "r") as f:
    API_KEY = json.load(f)["MONGODB_API_KEY_HACKATHON"]

url = "https://data.mongodb-api.com/app/data-sarkl/endpoint/data/v1/action/find"
collection_name = "hackathon"
database_name = "clslab-light-mixing"
payload = json.dumps(
    {
        "collection": collection_name,
        "database": database_name,
        "dataSource": "sparks-materials-informatics",
    }
)
headers = {
    "Content-Type": "application/json",
    "Access-Control-Request-Headers": "*",
    "api-key": API_KEY,
}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)


{"documents":[{"_id":"63a2a107e7351cef5622a76e","utc_timestamp":1671602430,"encrypted_device_id_truncated":"test","ch470":41,"_input_message":{"_session_id":"benchmark-dev-3c1e","B":0,"atime":14,"gain":32,"integration_time_s":0.01984921,"astep":475,"G":13,"R":22,"_experiment_id":"ec07496a-d1fd-4a24-b57d-874145451d9a"},"ch410":1,"ch440":7,"sd_card_ready":false,"ch510":10,"ch550":5,"ch670":1,"utc_time_str":"2022-12-21 06:00:30","onboard_temperature_K":296.9174,"ch620":26,"device_nickname":"CLSLab-light-public-test","ch583":13},{"_id":"63a2a10c068732fded64fbad","utc_timestamp":1671602441,"encrypted_device_id_truncated":"test","ch470":5634,"_input_message":{"_session_id":"benchmark-dev-3c1e","B":69,"atime":22,"gain":512,"integration_time_s":0.04533346,"astep":708,"G":44,"R":80,"_experiment_id":"a23d1443-6437-4e7f-8b7b-d95827efda6c"},"ch410":330,"ch440":5859,"sd_card_ready":false,"ch510":2414,"ch550":576,"ch670":460,"utc_time_str":"2022-12-21 06:00:41","onboard_temperature_K":294.5767,"ch62

In [9]:
data = json.loads(response.text)
documents = data["documents"]

In [11]:
len(documents)

1000

In [19]:
import pandas as pd

CHANNEL_NAMES = ["ch410", "ch440", "ch470", "ch510", "ch550", "ch583", "ch620", "ch670"]
INPUT_NAMES = ["R", "G", "B", "atime", "gain", "astep"]
OTHER_NAMES = ["utc_timestamp", "onboard_temperature_K"]
rows = []
for doc in documents:
    if "utc_timestamp" not in doc:
        continue
    msg = doc["_input_message"]
    other = [doc[key] for key in OTHER_NAMES]
    inputs = [msg[key] for key in INPUT_NAMES]
    sensor_data = [doc[ch] for ch in CHANNEL_NAMES]
    rows.append(other + inputs + sensor_data)
    
pd.DataFrame(rows, columns=OTHER_NAMES + INPUT_NAMES + CHANNEL_NAMES).to_csv(f"mongodb-data_{database_name}_{collection_name}.csv", index=False)

## Code Graveyard

In [ ]:
# from pymongo import MongoClient

# client = MongoClient("mongodb+srv://sgbaird:<password>@sparks-materials-inform.bgydt.mongodb.net/?retryWrites=true&w=majority")
# db = client.test